Theater Simulation model
CUNY SPS Data 604
Summer 2020

Created by: John Kellogg

In [1]:
import simpy 
import random
import statistics
from modsim import *
import matplotlib.pyplot as plt 

Defining the Global varibles 

In [2]:
def environ_var():
    global alpha
    global beta
    global wait_times
    global servers_array
    
    alpha = 3
    beta = 1.5
    wait_times = []
    servers_array = [1,2,3,4,5,6,7,8]

In [3]:
def plot_results(results):
    plt.hist(results, bins = 40)
    plt.show() 

Function to create the system object and ask for user input on numbers of staff per station

In [4]:
def make_system():
    
    environ_var()
    
    num_cashiers = input("Input # of cashiers working (1-4): ")
    num_servers = []
    num_ushers = 2
    num_health = 3
    mins = []
    secs = []                   
        
    return System(num_cashiers = num_cashiers, num_servers = num_servers, num_ushers = num_ushers, 
                  num_health = num_health, mins = mins, secs = secs)

Creating the theater environment and stations

In [5]:
class Theater(object):
     
    def __init__(self, env, num_cashiers, num_servers, num_ushers, num_health):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)
        self.health = simpy.Resource(env, num_health)

    # defining the process of tickets and waits of 1 to 3 minutes
    def purchase_ticket(self, moviegoer):
        yield self.env.timeout(random.paretovariate(alpha))

    # defining the Health check and waits 1 to 3 minutes    
    def health_check(self, moviegoer):
        yield self.env.timeout(random.paretovariate(alpha))

    # Add Random if possible
    def check_ticket(self, moviegoer):
        yield self.env.timeout(3 / 60)

    # defining the food options and waits of 1 to 5 minutes
    def sell_food(self, moviegoer):
        yield self.env.timeout(random.paretovariate(beta))

Function to build the movie going experience and build in wait times per station

In [6]:
def go_to_movies(env, moviegoer, theater):
    # Moviegoer arrives at the theater
    arrival_time = env.now
    snacks = random.choice([True, False])
    
    # getting a ticket
    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.purchase_ticket(moviegoer))
    
    # getting the health screening    
    with theater.health.request() as request:
        yield request
        yield env.process(theater.health_check(moviegoer))
    
    #getting ticket checked
    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))
    
    # random choice to buy snacks or not    
    if snacks == True:
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))
    
    # substract arrival time from wait times to get total time 
    wait_times.append(env.now - arrival_time)

Function to run the environment create the stations with the user defined staff numbers

In [7]:
def run_theater(env, num_cashiers, num_servers, num_ushers, num_health):
    theater = Theater(env, num_cashiers, num_servers, num_ushers, num_health)
    
    lambd = 3
    n = random.randint(1, 10)
    #start sim with people waiting for opening
    #replace with Random number
    for moviegoer in range(n):
        env.process(go_to_movies(env, moviegoer, theater))
    
    # wait time for next person
    while True:
        
        yield env.timeout(random.expovariate(lambd))
        
        # add another movie goer to sim 
        # replace with random number generator of 1-5
        moviegoer += n
        env.process (go_to_movies(env,moviegoer,theater))

Function to create the average (median) wait times

In [8]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

We always need a method to clear the model to run it again.

In [9]:
def reset_model():
    global wait_times
    global alpha
    global beta
    wait_times = [] 
    alpha = []
    beta = []
    system.num_cashiers = []

Function to plot results with the findings of the model

In [10]:
def plot_results(results):
    # results being the return value of the main model
    plt.hist(results, bins = 40)
    plt.show()

Main Function to run all active all the functions previously defined and print the results.

In [11]:
def run_simulation(system, num_servers):

   # Setup environment
    environ_var()
    random.seed(42)
    num_cashiers = int(system.num_cashiers) 
    num_servers = num_servers 
    num_ushers = int(system.num_ushers)
    num_health = int(system.num_health)

    # Run the simulation
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers, num_health))
    env.run(until=90)

    # View the results
    
    """
    for i in range(num_runs):
        results[i] = wait_times
        
    results_full = DataFrame(results)
    results_full = get_average_wait_time(results_full['0'])
    
    return results_full
    """
    get_average_wait_time(wait_times)
    #print_results()

In [12]:
def server_model(system):
 
    sweep = SweepSeries()
    
    for servers in servers_array:
        run_simulation(system, servers)
        sweep[servers] = get_average_wait_time(wait_times)

    return sweep

In [13]:
system = make_system()

Input # of cashiers working (1-4): 2


In [14]:
servers = server_model(system)

In [16]:
servers

,values
1,"(31.0, 4.0)"
2,"(32.0, 35.0)"
3,"(28.0, 32.0)"
4,"(26.0, 34.0)"
5,"(26.0, 34.0)"
6,"(26.0, 34.0)"
7,"(26.0, 34.0)"
8,"(26.0, 34.0)"


In [15]:
#reset_model()